In [17]:
'''
Details for LRW dataset:
- 500 words (check lrw_list.txt)
- 800-1000 train videos per word
- 50 test and 50 validation videos per word
- Split into train, test, and validation sets
- All videos are 29 frames long (1.16 seconds)
- Word occurs roughly in the middle

TODO: (update as completed)
- Switch labels to numeric or one-hot (currently strings)
- Experiement with transformations of the data
- Experiement with trimming the video to cut out unneeded words, and reduce number of frames
'''

import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
import pytorchvideo #look up how to use this
import cv2
import math
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim


NUM_FRAMES = 29 # Make sure to set all new videos to this length
FPS = 25
TIME = 1.16


In [26]:
class LipReadDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        """
        Initialize the dataset.
        Args:
            video_paths (list): List of file paths to video files.
            labels (list): List of corresponding labels.
            transform (callable, optional): Optional data transformations (e.g., resizing, normalization).
        """
        # self.word_folders = 
        # print(self.word_folders)
        self.frames = []
        self.labels = []
        for word_folder in os.listdir(root_dir):
            set_path = os.path.join(root_dir, word_folder, split)
            for video_file in os.listdir(set_path):
                video_path = os.path.join(set_path, video_file)
                if video_path.endswith('.txt'):
                    continue
                curr_frames = self.extract_frames(video_path)
                self.labels.append(word_folder)
                self.frames.append(curr_frames)
                print(f"added {video_path} to {split} set")
        self.transform = transform
        assert len(self.frames) == len(self.labels)

    def __len__(self):
        """Return the total number of samples in the dataset."""
        return len(self.frames)

    def __getitem__(self, idx):
        """
        Get a specific sample from the dataset.
        Args:
            idx (int): Index of the sample.
        Returns:
            video (tensor): Video frames (e.g., as a sequence of images).
            label (int): Corresponding label.
        """

        # Apply any necessary transformations (e.g., resizing, normalization)
        # if self.transform:
        #     video = self.transform(video)

        return self.frames[idx], self.labels[idx]
    
    def extract_frames(self, video_path, duration=1.16):
        # Read the video
        cap = cv2.VideoCapture(video_path)
        # EXPERIEMENT WITH THIS
        # duration_frames = int(duration * FPS)
        # end_frame = NUM_FRAMES -  9 # skip the last 9 frames
        # start_frame = end_frame - duration_frames # get middle frames
        frames = []
        # cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                # if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
                #     break
                frame_cropped = frame[120:200, 70:200] # Crop the frame
                frame_gray = cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2GRAY)
                frames.append(torch.from_numpy(frame_gray))
            else:
                break
        cap.release()
        return frames






In [27]:
transform = transforms.Compose([]) # Add any necessary transformations here

root_dir = 'lipread_mp4'
# trainset = LipReadDataset(root_dir, split='train', transform=transform)
print("finished trainset")
testset = LipReadDataset(root_dir, split='test', transform=transform)
print("finished testset")
valset = LipReadDataset(root_dir, split='val', transform=transform)
print("finished valset")

finished trainset
added lipread_mp4\ABOUT\test\ABOUT_00001.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00002.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00003.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00004.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00005.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00006.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00007.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00008.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00009.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00010.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00011.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00012.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00013.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00014.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00015.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00016.mp4 to test set
added lipread_mp4\ABOUT\test\ABOUT_00017.mp4 to test set
added lipread

In [49]:
# video_path = testset.frames[0]

# print(len(video_path))

# def loop_video(frames, loop_count):    
#     # Loop the video
#     for _ in range(loop_count):
#         for frame in frames:
#             cv2.imshow('frame', frame.numpy())
#             if cv2.waitKey(25) & 0xFF == ord('q'):
#                 break  # Break the loop if 'q' is pressed
#     cv2.destroyAllWindows()

# loop_video(video_path, 10)

#send to dataloader
batch_size = 32
# train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=True)

# dataiter = iter(train_loader)

In [47]:
# # TESTING IMAGE SIZE FOR CROP
# video_path = trainset.video_paths[2]

# def loop_video(video_path, loop_count, duration = 0.27):
#     # Read the video
#     cap = cv2.VideoCapture(video_path)
#     duration_frames = round(duration * FPS)
#     end_frame = NUM_FRAMES -  9 # skip the last 9 frames
#     start_frame = end_frame - duration_frames - 3  # get middle frames
#     print(end_frame, start_frame)
#     frames = []
#     cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
#     # Loop the video
#     frame_count = 0
#     for _ in range(loop_count):
#         cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # Set frame position to the beginning
#         while cap.isOpened():
#             ret, frame = cap.read()
#             if ret:
#                 frame_count += 1
#                 if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
#                     break
#                 frame_cropped = frame[120:200, 70:200]
#                 cv2.imshow('Looped Video', frame_cropped)
#                 if cv2.waitKey(25) & 0xFF == ord('q'):
#                     break
#             else:
#                 break
#     print(frame_count)
    
#     # Release video capture
#     cap.release()
#     cv2.destroyAllWindows()

# loop_video(video_path, 10)




20 10
100


In [93]:
class My3DCNN(nn.Module):
    def __init__(self, num_frames, height, width, channels, num_classes):
        super(My3DCNN, self).__init__()
        # input is [1x32x29x80x130]
        self.conv1 = nn.Conv3d(in_channels=channels, out_channels=32, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv3 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        # Calculate the shape of the flattened output from convolutional layers
        self.flatten_size = 128 * (num_frames // 4) * (height // 8) * (width // 8)

        self.fc1 = nn.Linear(self.flatten_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        print(x.shape)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        # Flatten the output
        x = x.view(-1, self.flatten_size)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x





In [30]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)



cuda:0


In [96]:
model = My3DCNN(num_frames=NUM_FRAMES, height=80, width=130, channels=1, num_classes=500).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [98]:
def train(model, train_loader, optimizer, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            
            inputs, labels = data
            # print(inputs[0].shape)
            inputs = torch.stack(inputs, dim=1).float().to(device)
            inputs = inputs.unsqueeze(1)
            print(inputs.shape)
            labels = torch.tensor(labels).to(device)
            # labels = labels.to(device)

            
            optimizer.zero_grad()

            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            #print
            running_loss += loss.item()
            if i % 10 == 9:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 10}')
                running_loss = 0.0
            
    print('Finished Training')




def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with nn.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = torch.stack(inputs, dim=1).float().to(device)
            inputs = inputs.unsqueeze(1)
            labels = torch.tensor(labels).to(device)
            
            outputs = model(inputs)
            _, predicted = nn.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')



In [99]:
train(model, val_loader, optimizer, criterion, num_epochs=5)
test(model, test_loader)

torch.Size([32, 1, 29, 80, 130])


ValueError: too many dimensions 'str'